In [1]:
!unzip -o /home/aistudio/data/data232024/ChatGPT生成文本检测器公开数据-更新.zip -d /home/aistudio/data # 解压原始数据集
!cp /home/aistudio/data/data233414/* /home/aistudio/work/ # 复制模型参数文件

Archive:  /home/aistudio/data/data232024/ChatGPT生成文本检测器公开数据-更新.zip
   creating: /home/aistudio/data/ChatGPT生成文本检测器公开数据-更新/
  inflating: /home/aistudio/data/ChatGPT生成文本检测器公开数据-更新/sample_submit.csv  
  inflating: /home/aistudio/data/ChatGPT生成文本检测器公开数据-更新/test.csv  
  inflating: /home/aistudio/data/ChatGPT生成文本检测器公开数据-更新/train.csv  


In [2]:
import numpy as np # 数值计算
import pandas as pd # 数据分析
from tqdm import tqdm # 进度条显示
import paddle # PaddlePaddle 深度学习框架
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer # 飞桨自然语言处理工具包（模型、分词器）
from paddle.io import DataLoader # 数据加载器
from paddlenlp.datasets import MapDataset # 数据集转换
from sklearn.model_selection import train_test_split # 训练集与验证集拆分
import matplotlib.pyplot as plt # 绘图

/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [3]:
data = pd.read_csv("/home/aistudio/data/ChatGPT生成文本检测器公开数据-更新/train.csv") # 加载赛事提供的训练数据
test_data = pd.read_csv("/home/aistudio/data/ChatGPT生成文本检测器公开数据-更新/test.csv") # 加载赛事所需提交的测试数据
data.sample(frac=1).head() # 随机查看 5 行训练数据中的内容

,name,label,content
7365,7366,1,[1914 1908 778 1279 5051 5212 329 1570 4253 ...
886,887,0,[1564 1105 1254 1241 751 4266 1759 4294 5212 ...
6587,6588,0,[ 199 998 3635 177 3125 2593 2483 2669 1424 ...
7789,7790,1,[ 248 3094 3874 169 1894 1782 506 2214 5212 ...
13526,13527,1,[2177 3169 2417 3125 2178 672 5212 304 253 ...


In [4]:
# 载入模型与分词器

# 使用 ernie-3.0-mini-zh 序列分类模型，并将分类类别数设置为 2
model = AutoModelForSequenceClassification.from_pretrained("ernie-3.0-mini-zh", num_classes=2)
# 使用 ernie-3.0-mini-zh 分词器
tokenizer = AutoTokenizer.from_pretrained("ernie-3.0-mini-zh")

# 定义 AdamW 优化器，学习率为 0.000001
optimizer = paddle.optimizer.AdamW(1e-5, parameters=model.parameters())

# 定义损失函数为交叉熵函数，计算每个 mini batch 的均值
loss_fn = paddle.nn.loss.CrossEntropyLoss(reduction='mean')

[2023-08-08 09:26:10,501] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load 'ernie-3.0-mini-zh'.
[2023-08-08 09:26:10,504] [    INFO] - Model config ErnieConfig {
  "attention_probs_dropout_prob": 0.1,
  "enable_recompute": false,
  "fuse": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "ernie",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "paddlenlp_version": null,
  "pool_act": "tanh",
  "task_id": 0,
  "task_type_vocab_size": 16,
  "type_vocab_size": 4,
  "use_task_id": true,
  "vocab_size": 40000
}

[2023-08-08 09:26:10,506] [    INFO] - Configuration saved in /home/aistudio/.paddlenlp/models/ernie-3.0-mini-zh/config.json
[2023-08-08 09:26:10,508] [    INFO] - Downloading ernie_3.0_mini_zh.pdparams from https://bj.bceb

In [5]:
# 按照 10% 的比例划分训练集与验证集
train_data, valid_data = train_test_split(data, test_size=0.1)

# 下面就是一堆操作，把数据变成数据加载器可以识别的格式，自定义数据集类也是同样的效果
train_dict = train_data.to_dict(orient='records')
valid_dict = valid_data.to_dict(orient='records')
train_ds = MapDataset(train_dict)
valid_ds = MapDataset(valid_dict)

# 将整体数据拆分为 30 份，放入数据加载器，就是一次性会有 <总样本数 / 30> 份数据同时并行计算，份数越多，并行越多，显存占用越大，需要根据需求来选择
train_loader = DataLoader(train_dict, batch_size=60, shuffle=True) # 训练数据可以随机打乱，让模型更好地学习，减轻学习到无关特征的问题
valid_loader = DataLoader(valid_dict, batch_size=60) # 这里用的是 V100 32G，如果是显存更小的卡，需要调小一点，不然会炸显存

In [6]:
epochs=200
best_accuracy=0
for epoch in range(epochs): # 训练 30 轮
    # 训练过程
    model.train() # 切换模型为训练模式
    for batch_x in tqdm(train_loader): # 每次从数据加载器读入一批(batch)数据
        X = tokenizer(batch_x["content"], max_length=1015, padding=True) # 将数据转换为模块可处理的数据形式
        input_ids = paddle.to_tensor(X['input_ids'], dtype="int32") # 将 input_ids 变为张量，方便并行计算
        token_type_ids = paddle.to_tensor(X['token_type_ids'], dtype="int32") # 将 token_type_ids 变为张量
        pred = model(input_ids, token_type_ids) # 将数据读入模型，并得到计算后的结果
        loss = loss_fn(pred, paddle.to_tensor(batch_x["label"], dtype="int32")) # 对比预测结果与真实结果，计算损失函数的值
        loss.backward() # 反向传播，计算梯度
        optimizer.step() # 优化器根据梯度与学习率调整模型参数
        optimizer.clear_gradients() # 清空梯度，避免下次梯度计算时累加

    # 验证过程
    model.eval() # 切换模型为验证模式
    val_loss = [] # 验证集数据的损失函数合集
    with paddle.no_grad(): # 在模型验证时，只做前向计算，因此不需要保存梯度信息
        for batch_x in tqdm(valid_loader): # 下面的操作与训练过程相同
            X = tokenizer(batch_x["content"], max_length=1015, padding=True)
            input_ids = paddle.to_tensor(X['input_ids'], dtype="int32")
            token_type_ids = paddle.to_tensor(X['token_type_ids'], dtype="int32")
            pred = model(input_ids, token_type_ids)
            loss = loss_fn(pred, paddle.to_tensor(batch_x["label"], dtype="int32"))
            val_loss.append(loss.item()) # 将计算出的损失函数值存入合集
            
    # 打印本轮训练的验证集损失函数值，与预测正确率
    accuracy = (pred.argmax(1) == batch_x["label"]).astype('float').mean().item()
    print('Epoch {0}, Val loss {1:3f}, Val Accuracy {2:3f}'.format(
    epoch,
    np.mean(val_loss), 
    (pred.argmax(1) == batch_x["label"]).astype('float').mean().item()
))
    if accuracy>best_accuracy:
        paddle.save(model.state_dict(),"/home/aistudio/work/model_best.pdparams")
        paddle.save(optimizer.state_dict(), "/home/aistudio/work/optimizer_best.pdopt")
        best_accuracy=accuracy

paddle.save(model.state_dict(),"/home/aistudio/work/model.pdparams")
paddle.save(optimizer.state_dict(), "/home/aistudio/work/optimizer.pdopt")

100%|██████████| 24/24 [00:06<00:00,  3.76it/s]


Epoch 0, Val loss 0.182319, Val Accuracy 0.950000


100%|██████████| 24/24 [00:06<00:00,  3.61it/s]


Epoch 1, Val loss 0.154067, Val Accuracy 1.000000


100%|██████████| 24/24 [00:06<00:00,  3.78it/s]


...

Epoch 197, Val loss 0.050068, Val Accuracy 1.000000


100%|██████████| 24/24 [00:06<00:00,  3.74it/s]


Epoch 198, Val loss 0.099612, Val Accuracy 1.000000


100%|██████████| 24/24 [00:06<00:00,  3.74it/s]

Epoch 199, Val loss 0.048090, Val Accuracy 1.000000


**迭代时注意更新条件**

In [7]:
# 如果你拿到了模型参数（在 AIStudio 中提供），你可以运行这行代码，如果直接运行模型，则没有必要运行

# 载入模型参数、优化器参数的最后一个epoch保存的检查点
layer_state_dict = paddle.load("/home/aistudio/work/model_best.pdparams")
opt_state_dict = paddle.load("/home/aistudio/work/optimizer_best.pdopt")

# 将加载后的参数与模型关联起来
model.set_state_dict(layer_state_dict)
optimizer.set_state_dict(opt_state_dict)

In [8]:
def infer(string: str) -> int:
    """将文本传入模型并返回预测结果
    
    输入：
        - string: str
            待预测的文本内容
    
    输出:
        - result: int
            模型的预测结果
    """
    X = tokenizer([string], max_length=1015, padding=True)
    input_ids = paddle.to_tensor(X['input_ids'], dtype="int32")
    token_type_ids = paddle.to_tensor(X['token_type_ids'], dtype="int32")
    pred = model(input_ids, token_type_ids)
    result = pred.argmax(1).item() # 获取预测概率最大的那个类别
    return result

In [9]:
test_data["label"] = test_data["content"].apply(infer) # 将测试集的每个文本送入模型返回结果
submit = test_data.drop(columns=["content"]) # 生成提交数据（就是把带结果的测试集丢掉内容，复制一份）
submit.to_csv("submit3.csv", index=False) # 保存 CSV 文件